In [1]:
import pandas as pd
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy import select
from transformers import HotelDataReadCsv, HotelDataEncoder
from ssrs import HotelData

In [2]:
db = Database()

hotel_mapping = pd.read_sql(
    sql=select(
        db.mapping_hotel.c.external_code,
        db.mapping_hotel.c.external_name,
        db.mapping_hotel.c.hotel_id,
    ),
    con=db.engine.connect(),
)

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
pipeline = Pipeline(
    [
        ("csv_reader", HotelDataReadCsv()),
        ("data_encoder", HotelDataEncoder()),
    ]
)

In [4]:
dfs = []

for destination in ["DU", "OM"]:
    data = HotelData(destination).get()
    df = pipeline.fit_transform(data)

    if destination == "DU":
        df = df[df["Country"].isin(["AE", "SA"])]

    dfs.append(df)

hotel_list = pd.concat(dfs, axis=0, ignore_index=True)

In [5]:
merged_list = hotel_list.merge(
    hotel_mapping, how="left", left_on="HotelID", right_on="external_code"
)
merged_list["Comparison"] = merged_list["HotelName_GWG"] == merged_list["external_name"]

In [6]:
merged_list.to_csv("hotel_list.csv", index=False)